In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [4]:
# create arangoDB database, or open it if it already exists
try:
    corp_db = conn.createDatabase(name="corp_data")
    # from command line:
    # arangoimport --file '''path to CorpData.csv''' --collection corp_data --create-collection true --type csv --server.database corp_data
    # arangoimport --file '''path to CorpIndividualExport.csv''' --collection corp_individual --create-collection true --type csv --server.database corp_data

except:
    corp_db = conn["corp_data"]

In [ ]:
# arangoDB database structure:

# corp_db: arangoDB database
# corp_db["corp_data"]: arangoDB collection
# corp_db["corp_data"][17777999]: arangoDB document (one piece of data, or one corporation)

In [6]:
# extracting the values of the indiv_data_processed collection
indiv_aql = "FOR x IN indiv_data_processed RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')
indiv_ar = indiv.to_numpy()

In [8]:
# extracting the keys of the l3_assess collection
key_aql = "FOR x IN indiv_data_processed RETURN x._key"
key_query_result = corp_db.AQLQuery(key_aql,rawResults=True, batchSize = 1000)
indiv_keys = []

for key in key_query_result:
    indiv_keys.append(key)

In [8]:
def indiv2indiv(keys, batch_size):
    
    for i in range(0,100):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_keys = keys[i*batch_size:(i+1)*batch_size][:]
            
        for key in range(0,batch_keys.shape[0]):
            indiv = corp_db["indiv_data_processed"].fetchDocument(key)
            id1 = "indiv_data_processed/" + key
            for i_id in indiv_id:
                edge_attributes = {"_from": corp_id, "_to": i_id, "type": 'corp2indiv', "DataID": cid}
                edge = corp_db["indiv2indiv"].createDocument(edge_attributes)
                edge.save()

indiv2indiv(indiv_keys,1000)

1200
1200000 1201000
1201
1201000 1202000
1202
1202000 1203000
1203
1203000 1204000
1204
1204000 1205000
1205
1205000 1206000
1206
1206000 1207000
1207
1207000 1208000
1208
1208000 1209000
1209
1209000 1210000
1210
1210000 1211000
1211
1211000 1212000
1212
1212000 1213000
1213
1213000 1214000
1214
1214000 1215000
1215
1215000 1216000
1216
1216000 1217000
1217
1217000 1218000
1218
1218000 1219000
1219
1219000 1220000
1220
1220000 1221000
1221
1221000 1222000
1222
1222000 1223000
1223
1223000 1224000
1224
1224000 1225000
1225
1225000 1226000
1226
1226000 1227000
1227
1227000 1228000
1228
1228000 1229000
1229
1229000 1230000
1230
1230000 1231000
1231
1231000 1232000
1232
1232000 1233000
1233
1233000 1234000
1234
1234000 1235000
1235
1235000 1236000
1236
1236000 1237000
1237
1237000 1238000
1238
1238000 1239000
1239
1239000 1240000
1240
1240000 1241000
1241
1241000 1242000
1242
1242000 1243000
1243
1243000 1244000
1244
1244000 1245000
1245
1245000 1246000
1246
1246000 1247000
1247
1247000 

In [7]:
print(indiv_ar.shape[0]/1000)

4905.639


In [ ]:
def one_hot(indices,df):
    hot = np.full(len(df),False)
    hot[indices] = True
    cold = np.invert(hot)
    return hot, cold

def corp2indiv_test(corp_df, indiv_df):
    print(len(indiv_df))
    for j in range(0,corp_df.shape[0]):
        cid = corp_df[j][3] 
        corp_id = corp_df[j][1]
        c_indiv = np.where(indiv_df[:,3] == cid)[0]
        #print(c_indiv)
        #mask, invert_mask = one_hot(c_indiv,indiv_df)
        #print(mask)
        indiv_id = indiv_df[c_indiv][:,1]
        #indiv_id = indiv_df[mask,1]
        for i_id in indiv_id:
          print(corp_id,i_id)

        #indiv_df = indiv_df[invert_mask,:]
    #print(len(indiv_df))

corp2indiv_test(corp_ar[0:100,:],indiv_ar)

In [ ]:
print(corp_ar[0:10,:])

In [ ]:
corp2indiv2(corp_ar,indiv_ar,1000)

0
1 1000


KeyboardInterrupt: ignored

In [ ]:
corp2 = corp.loc[0:0,:]

In [ ]:
print(len(indiv))

4905639
